In [ ]:
!git clone https://github.com/mcity/mcity_data_engine.git

In [ ]:
!cp -R mcity_data_engine/* .

In [ ]:
!pip install -r requirements_colab.txt

In [ ]:
!export HF_HUB_ETAG_TIMEOUT=5000
!export HF_HUB_DOWNLOAD_TIMEOUT=1000
!python main.py

In [ ]:
import fiftyone as fo
samples_panel = fo.Panel(type="Samples", pinned=True)

embeddings_panel = fo.Panel(
    type="Embeddings",
    state=dict(brainResult="clip_vit_base32_torch_umap", colorByField="metadata.size_bytes"),
)

spaces = fo.Space(
    children=[
        fo.Space(
            children=[
                fo.Space(children=[samples_panel]),
            ],
            orientation="horizontal",
        ),
        fo.Space(children=[embeddings_panel]),
    ],
    orientation="vertical",
)
fo.launch_app(dataset=fo.load_dataset('fisheye8k'),spaces=spaces)